In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import os
from pytorch import *
from mikkel import *

In [ ]:
with open("train.pkl", "rb") as f:
    df = pickle.load(f)
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_vec2', 'y_vec2', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()

x = torch.Tensor(x)
y = torch.Tensor(y)
train = TensorDataset(x,y)
train_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [ ]:
with open("val.pkl", "rb") as f:
    df = pickle.load(f)
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_vec2', 'y_vec2', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()

x = torch.Tensor(x)
y = torch.Tensor(y)
val = TensorDataset(x,y)
val_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
input_dim = 8
output_dim = 2
hidden_dim = 64
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 10
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'dropout_prob' : dropout}

model = get_model('LSTM', model_params).to(device)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
opt.plot_losses()

# predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)

In [ ]:
# torch.version.cuda
# torch.cuda.is_available()
# torch.cuda.current_device()
# torch.cuda.get_device_name()

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader

import numpy as np

import matplotlib.pyplot as plt
from PIL import Image, ImageOps

from mikkel import *
import pickle

In [ ]:
with open("train.pkl", "rb") as f:
    df = pickle.load(f)
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_vec2', 'y_vec2', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()

# x[:,::2] /= 1280
# x[:,1::2] /= 720
# y[:,0] /= 1280
# y[:,1] /= 720

x = torch.Tensor(x)
y = torch.Tensor(y)

train = TensorDataset(x,y)
train_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using {device} device')

In [ ]:
model = Mikkel().to(device)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)

for t in range(300):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
#     if t % 2000 == 1999:
#         print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(t)

# print(f'Result: {model.string()}')

In [ ]:
def calc_next(x, y_pred):
    next = torch.tensor([[
        x[0] + y_pred[0],    # x_pos
        x[1] + y_pred[1],    # y_pos
        y_pred[0],           # x_vec
        y_pred[1],           # y_vec
        x[2],                # x_vec2
        x[3],                # y_vec2
        x[-2],               # x_dest
        x[-1]                # y_dest
    ]])
    return next

def simulate(model, inputs, iterations):
    for i in range(iterations):
        y_pred = model(inputs[i:])
        next = calc_next(inputs[i], y_pred[0])
        inputs = torch.cat((inputs, next))

In [ ]:
model(x)

In [ ]:
tis = torch.tensor([
        [[1050, 275, 0, 0, 0, 0, 1000, 470]],
        [[580*2, 50*2, 0, 0, 0, 0, 250*2, 280*2]],
        [[200*2, 225*2, 0, 0, 0, 0, 500*2, 250*2]],
        [[125*2, 200*2, 0, 0, 0, 0, 460*2, 50*2]],
        [[580*2, 50*2, 0, 0, 0, 0, 250*2, 300*2]],
        [[150*2, 240*2, 0, 0, 0, 0, 250*2, 300*2]]
        ]).type(torch.FloatTensor)

# tis[:,::2] /= 1280
# tis[:,1::2] /= 720

fig, ax = plt.subplots(figsize=(20,20))
im = Image.open("intersection2.png")
im = ImageOps.flip(im)
ax.set_xlim(0,1280)
ax.set_ylim(0,720)
ax.imshow(im, origin='lower')

inputs = tis[5]
for i in range(100):
    y_pred = model(inputs[i:])
    next = calc_next(inputs[i], y_pred[0])
    inputs = torch.cat((inputs, next))

ax.scatter(inputs[:,0], inputs[:,1])